## import transformers and create the model and tokenizers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load pretrained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")



# Create a partial sentence and tokenize it

text = "Madrid es una ciudad maravillosa y"
inputs = tokenizer(text,return_tensors="pt")

# show tokens as numbers
inputs["input_ids"]

### Check the tokenization



In [ ]:
import pandas as pd

def show_tokenization(inputs):
    return pd.DataFrame([(id,tokenizer.decode(id)) for id in inputs["input_ids"][0]],
        columns=["id","token"],
    )
show_tokenization(inputs)

## now we copute the probability of the neext token

In [ ]:
import torch

# torch.no_grad() -> disables gradient computation during inference
# this reuce the memory ussage and speed up execution
with torch.no_grad():
    # **inputs unpacks dictionary formatted model inputd
    # slicing [:.-1,:] extracts last token logits from sequence output
    logits = model(**inputs).logits[:,-1,:]
    probabilities = torch.nn.functional.softmax(logits[0],dim=-1)

def show_next_token_choices(probabilities,top_n=5):
    
  
    return pd.DataFrame(
        [(id,tokenizer.decode(id),p.item()) for id,p in enumerate(probabilities) if p.item()],columns=["id","token","p"],).sort_values("p",ascending=False)[:top_n]

show_next_token_choices(probabilities)